In [1]:
import requests
import json
import time


In [2]:
myaddress = 'TW7eKyRw8ow98fpFibLe84qKwSbrvuD1KB' #no txn in prod so far
address_example = 'TUD4YXYdj2t1gP5th3A7t97mx1AUmrrQRt'
address_DEX = 'TAUN6FwrnwwmaEqYcckffC7wYmbaS6cBiX'
trc20_USDT = 'TR7NHqjeKQxGTCi8q8ZY4pL8otSzgjLj6t'

In [29]:
def get_latest_block():
    #return the latest block number and ts
    url = "https://api.trongrid.io/v1/blocks/latest/events"
    response = requests.request("GET", url)

    block_number = json.loads(response.text)['data'][0]['block_number']
    block_ts = json.loads(response.text)['data'][0]['block_timestamp']
    
    return block_number, block_ts

get_latest_block()

(28125045, 1614764295000)

# 1.根据钱包地址

#### 获取账户详情

In [3]:
url_get_account = 'https://api.trongrid.io/v1/accounts/{address}'.format(address = address_example)
response = requests.request("GET", url_get_account)
print(json.loads(response.text)['data'][0].keys())

dict_keys(['owner_permission', 'account_resource', 'active_permission', 'address', 'create_time', 'latest_consume_time', 'allowance', 'latest_opration_time', 'free_asset_net_usageV2', 'is_witness', 'free_net_usage', 'assetV2', 'balance', 'trc20', 'latest_consume_free_time', 'account_name', 'latest_withdraw_time', 'witness_permission'])


#### 获取账户的历史交易

In [4]:
url_get_account_txn = 'https://api.trongrid.io/v1/accounts/{address}/transactions'.format(address = address_DEX)
response = requests.request("GET", url_get_account_txn)
print(json.loads(response.text)['data'][0])

{'ret': [{'contractRet': 'SUCCESS', 'fee': 4196640}], 'signature': ['bb3a8e124c31d7d1245fe46cf061eb02623042b81aab5e4b63223ab894ecf53e0bf43abb90c262ea4a2c81ef70b05235771c152d968b76750e4d37484374d87d00'], 'txID': '619fa67091522724cc1df0a39c278ccac614303d797f48562362f0d5733f21db', 'net_usage': 0, 'raw_data_hex': '0a0222f3220878d02d34fec8139b40a0eefeb9ff2e5aae01081f12a9010a31747970652e676f6f676c65617069732e636f6d2f70726f746f636f6c2e54726967676572536d617274436f6e747261637412740a15410583a68a3bcd86c25ab1bee482bac04a216b0261121541a614f803b6fd780986a42c78ec9c7f77e6ded13c2244a9059cbb000000000000000000000041ece7458cafe362e7f707f5105d4f7add6a34ffe700000000000000000000000000000000000000000000000000000000299a061d70dba3fbb9ff2e900180ade204', 'net_fee': 48300, 'energy_usage': 0, 'blockNumber': 28123892, 'block_timestamp': 1614760827000, 'energy_fee': 4148340, 'energy_usage_total': 29631, 'raw_data': {'contract': [{'parameter': {'value': {'data': 'a9059cbb000000000000000000000041ece7458cafe362e7f707f51

#### 获取账户在某一合约下的历史交易

In [5]:
# 增加以下过滤条件，只获取trc20-USDT相关事件
url_get_account_txn_trc20 = 'https://api.trongrid.io/v1/accounts/{address}/transactions/trc20'.format(address = address_DEX)
querystring = {"contract_address":"TR7NHqjeKQxGTCi8q8ZY4pL8otSzgjLj6t"}
response = requests.request("GET", url_get_account_txn_trc20, params=querystring)
print(json.loads(response.text)['data'][0])

{'transaction_id': '619fa67091522724cc1df0a39c278ccac614303d797f48562362f0d5733f21db', 'token_info': {'symbol': 'USDT', 'address': 'TR7NHqjeKQxGTCi8q8ZY4pL8otSzgjLj6t', 'decimals': 6, 'name': 'Tether USD'}, 'block_timestamp': 1614760827000, 'from': 'TAUN6FwrnwwmaEqYcckffC7wYmbaS6cBiX', 'to': 'TXZqXNM97wn9ccGPFxtqW9aLuCJk3SU2vN', 'type': 'Transfer', 'value': '697959965'}


# 2. 根据合约地址

### 获取合约历史事件（暂无websocket订阅方法，只能GET）

In [16]:
url_get_contract_txn = "https://api.trongrid.io/v1/contracts/{c_address}/transactions".format(c_address = trc20_USDT)
#限制返回结果的最小区块时间戳，以及每页的交易属（默认为20，最大为200）
querystring = {"min_block_timestamp":"1614761300000", "limit":"200"} 
response = requests.request("GET", url_get_contract_txn, params = querystring)
print(json.loads(response.text))

{'data': [{'ret': [{'contractRet': 'SUCCESS'}], 'signature': ['ca7c8178bd461f9e6413452e9625900a0848eb72cb50a099b15c558ff46edfc119b8346efb368acab4a594583970ecc90377dff6c5faf323660fe4eb1b42a8f101'], 'txID': '49315a2c40da4c870a50806f1e174f1fe09fc0a687188037ab200d658cdbd9c4', 'net_usage': 345, 'raw_data_hex': '0a0223f822088a427c70afbaf97040e090b2baff2e5aae01081f12a9010a31747970652e676f6f676c65617069732e636f6d2f70726f746f636f6c2e54726967676572536d617274436f6e747261637412740a1541b4ff7a947255c843522d0a42f11385ec419b1df6121541a614f803b6fd780986a42c78ec9c7f77e6ded13c2244a9059cbb0000000000000000000000000233709379d6fd2797f44b9cbee460d18992015c000000000000000000000000000000000000000000000000000000012a077c2470d2cbaebaff2e900180c2d72f', 'net_fee': 0, 'energy_usage': 14631, 'block_timestamp': '1614761667000', 'blockNumber': '28124171', 'energy_fee': 0, 'energy_usage_total': 14631, 'raw_data': {'contract': [{'parameter': {'value': {'data': 'a9059cbb0000000000000000000000000233709379d6fd2797f44b9cbee46

In [19]:
#估算:最新的trc20-USDT交易  大约分布在10个区块中（大约需要三秒钟产生一个新的区块）
set([i['block_timestamp'] for i in json.loads(response.text)['data']])

{'1614761634000',
 '1614761637000',
 '1614761640000',
 '1614761643000',
 '1614761649000',
 '1614761652000',
 '1614761655000',
 '1614761658000',
 '1614761661000',
 '1614761664000',
 '1614761667000'}

In [35]:
url_get_contract_events = "https://api.trongrid.io/v1/contracts/{c_address}/events".format(c_address = trc20_USDT)
#限制返回结果的最小区块时间戳，以及每页的交易属（默认为20，最大为200）
querystring = {"event_name":"Transfer","block_number":get_latest_block()[0],"limit":"200"}
response = requests.request("GET", url_get_contract_txn, params = querystring)
print(querystring['block_number'])
print(json.loads(response.text))


28125137
{'data': [{'ret': [{'contractRet': 'SUCCESS'}], 'signature': ['796b57baf18ec89b1a9d386525b6fb38ebb89431a7b3f94a6576a0aa37c9a31564b59f646de6a78e24685203de87f8470675f6131fd540f9672c7fc9b400985100'], 'txID': 'a915947e8c872949c4a96e8ff3ebe604cc130be3c49e60e70d99200d0b635275', 'net_usage': 345, 'raw_data_hex': '0a0226d22208e9716852a462e23140f8a4c6ccff2e5aae01081f12a9010a31747970652e676f6f676c65617069732e636f6d2f70726f746f636f6c2e54726967676572536d617274436f6e747261637412740a1541086830c67fdeeeb6bb51f4c0c02d2597374328af121541a614f803b6fd780986a42c78ec9c7f77e6ded13c2244a9059cbb0000000000000000000000414565fb2b4fd990c78ae3bcb78289eb68ffb3aec0000000000000000000000000000000000000000000000000000000000ae2dd8070f882b1bbff2e9001c096b102', 'net_fee': 0, 'energy_usage': 0, 'block_timestamp': '1614764571000', 'blockNumber': '28125137', 'energy_fee': 2048340, 'energy_usage_total': 14631, 'raw_data': {'contract': [{'parameter': {'value': {'data': 'a9059cbb0000000000000000000000414565fb2b4fd990c78a

In [37]:
len(json.loads(response.text)['data'])

200

In [41]:
#以mainnet区块blockNumber 28125137 为例，其中的USDT交易仅有12笔
len([i['blockNumber'] for i in json.loads(response.text)['data'] if i['blockNumber'] == str(querystring['block_number'])])

12

In [36]:
# 若声明每页的返回数为200且最新区块USDT交易不足200，则会返回之前的区块的USDT交易信息
set([i['blockNumber'] for i in json.loads(response.text)['data']])

{'28125129',
 '28125130',
 '28125131',
 '28125132',
 '28125133',
 '28125134',
 '28125135',
 '28125136',
 '28125137'}

### 获取某一代币所有持有者的余额信息

In [21]:
url_get_tec20_balances = "https://api.trongrid.io/v1/contracts/{c_address}/tokens".format(c_address = trc20_USDT)
querystring = {"limit":"200"} 
response = requests.request("GET", url_get_tec20_balances, params = querystring)
print(json.loads(response.text)['data'][:10])

[{'TMuA6YqfCeX8EhbfYEg5y7S4DqzSJireY9': '2500000000000000'}, {'TAUN6FwrnwwmaEqYcckffC7wYmbaS6cBiX': '1625246835257550'}, {'TNaRAoLUyYEV2uF7GUrzSjRQTU8v5ZJ5VR': '1114538033737597'}, {'TWd4WrZ9wn84f5x1hZhL4DHvk738ns5jwb': '700000000000000'}, {'TFsmkjujT9omG5TFrEXYbDocTdjeeHgcrQ': '600000007000124'}, {'TNMDwbHyAQo38tK4fCYe4ooSZXesbK8ExZ': '578000000000076'}, {'TM1zzNDZD2DPASbKcgdVoTYhfmYgtfwx9R': '440758301069313'}, {'TJyf3bMf5CXYe511EFPbN5dZ2W8edMnzsn': '420817281000042'}, {'TKHuVq1oKVruCGLvqVexFs6dawKv6fQgFs': '382181363328931'}, {'TXFBqBbqJommqZf7BV8NNYzePh97UmJodJ': '329928437497811'}]


# 3.创建交易 (deprecated)

In [ ]:
url = "https://api.shasta.trongrid.io/wallet/gettransactionsign"
acct1 = 'TW7eKyRw8ow98fpFibLe84qKwSbrvuD1KB'
acct2 = 'TF6omHyds48Q45FH3r5m2ePHuLaWNeGPVM'
pk2 = '53e77495002645aa50303d2ee82af50797618a7aaee5616f2509d40b45353656'
current_ts = int((time.time() * 1000) #GMT+0

In [55]:
url_create = 'https://api.trongrid.io/wallet/createtransaction'
payload_value = {'amount':1000,'owner_address':acct2,'to_address':acct1}
headers = {"Content-Type": "application/json"}

response = requests.request("POST", url, json=payload_value, headers=headers)
response.text

'{"Error":"class java.lang.NullPointerException : null"}\n'

In [45]:

payload = {
    "transaction": {
        "raw_data": {'contract':[{
                        'parameter':{'value': payload_value,
                                     'type_url':'type.googleapis.com/protocol.TransferContract'},
                                     'type':'TransferContract'}
                                ],
                     'ref_block_bytes':'5e4b',
                     'ref_block_hash':'47c9dc89341b300d',
                     'expiration':current_ts+60,
                     'timestamp':current_ts},
        "visible": True
    },
    "privateKey": pk2
}
headers = {"Content-Type": "application/json"}

response = requests.request("POST", url, json=payload, headers=headers)

print(response.text)

{"visible":true,"signature":["f7c7a633b4b98fad84336ec32ebc5860945cc6a363ca859a4e29506d079ad25e46415f872a2ae02431e3af4a9358b6bfbda76ae4b0694f0ce7d458f17e36db4900"],"txID":"b398c11e7b330b519021f37d5fb7a21f791b0fc025d3c60f1a6e8abfa87436ac","raw_data":{"contract":[{"parameter":{"value":{"amount":1000,"owner_address":"TF6omHyds48Q45FH3r5m2ePHuLaWNeGPVM","to_address":"TW7eKyRw8ow98fpFibLe84qKwSbrvuD1KB"},"type_url":"type.googleapis.com/protocol.TransferContract"},"type":"TransferContract"}],"ref_block_bytes":"5e4b","ref_block_hash":"47c9dc89341b300d","expiration":1591089627000,"timestamp":1591089567635},"raw_data_hex":"0a025e4b220847c9dc89341b300d40f8fed3a2a72e5a66080112620a2d747970652e676f6f676c65617069732e636f6d2f70726f746f636f6c2e5472616e73666572436f6e747261637412310a154138488f1eb01114f8f9ddd815593ba14295d5bc99121541dcfafc0b47708b04bdac37262893cdf091da5c2b18e8077093afd0a2a72e"}



In [ ]:

{"contract":[{"parameter":{"value":{"amount":1000,"owner_address":"41608f8da72479edc7dd921e4c30bb7e7cddbe722e","to_address":"41e9d79cc47518930bc322d9bf7cddd260a0260a8d"},"type_url":"type.googleapis.com/protocol.TransferContract"},"type":"TransferContract"}],"ref_block_bytes":"5e4b","ref_block_hash":"47c9dc89341b300d","expiration":1591089627000,"timestamp":1591089567635}
{"contract":[{"parameter":{"value":{"amount":1000,"owner_address":"41608f8da72479edc7dd921e4c30bb7e7cddbe722e","to_address":"41e9d79cc47518930bc322d9bf7cddd260a0260a8d"},"type_url":"type.googleapis.com/protocol.TransferContract"},"type":"TransferContract"}],"ref_block_bytes":"5e4b","ref_block_hash":"47c9dc89341b300d","expiration":1591089627000,"timestamp":1591089567635}

CreateAccount
在链上创建账号. 一个已经激活的账号创建一个新账号需要花费 0.1 TRX 或等值 Bandwidth.
[https://cn.developers.tron.network/reference#%E5%9C%B0%E5%9D%80%E5%92%8C%E5%B8%90%E6%88%B7]